In [5]:
import json
import pandas as pd
from collections import Counter

# JSON einlesen
with open("testdata/neo4j_data_politicians.json", 'r', encoding='utf-8') as f:
    politicians = json.load(f)

print(f"�� Gesamtanzahl Politiker: {len(politicians)}")

# 1. Datenstruktur-Validierung
print("\n🔍 DATENSTRUKTUR-VALIDIERUNG:")
expected_fields = ['detail_page', 'firstname', 'full_name', 'lastname', 'birth_year', 'death_year', 'neo4j_element_id', 'neo4j_content']

for field in expected_fields:
    missing_count = sum(1 for p in politicians if field not in p)
    print(f"  {field}: {len(politicians) - missing_count}/{len(politicians)} vorhanden")

# 2. Content-Verfügbarkeit
print("\n📊 CONTENT-VERFÜGBARKEIT:")
content_counts = [len(p.get('neo4j_content', [])) for p in politicians]
print(f"  Politiker mit Content: {sum(1 for c in content_counts if c > 0)}/{len(politicians)}")
print(f"  Durchschnitt Content pro Politiker: {sum(content_counts) / len(politicians):.1f}")
print(f"  Min/Max Content: {min(content_counts)} / {max(content_counts)}")

# 3. ID-Integrität
print("\n🆔 ID-INTEGRITÄT:")
element_ids = [p.get('neo4j_element_id') for p in politicians]
unique_ids = set(element_ids)
print(f"  Eindeutige IDs: {len(unique_ids)}/{len(politicians)}")
print(f"  Duplikate: {len(element_ids) - len(unique_ids)}")

# 4. Feld-Füllgrad pro Objekt
print("\n📋 FELD-FÜLLGRAD:")
field_completeness = {}
for field in expected_fields:
    if field == 'neo4j_content':
        # Content ist ein Array, prüfe ob nicht leer
        non_empty = sum(1 for p in politicians if p.get(field) and len(p[field]) > 0)
        field_completeness[field] = non_empty
    else:
        non_null = sum(1 for p in politicians if p.get(field) and p[field] is not None)
        field_completeness[field] = non_null

for field, count in field_completeness.items():
    percentage = (count / len(politicians)) * 100
    print(f"  {field}: {count}/{len(politicians)} ({percentage:.1f}%)")

# 5. Content-Qualität
print("\n📝 CONTENT-QUALITÄT:")
content_fields = ['content_id', 'section_header', 'section_content']
content_issues = []

for i, politician in enumerate(politicians):
    content = politician.get('neo4j_content', [])
    for j, content_node in enumerate(content):
        for field in content_fields:
            if field not in content_node or not content_node[field]:
                content_issues.append(f"Politiker {i}, Content {j}: {field} fehlt/leer")

print(f"  Content-Probleme gefunden: {len(content_issues)}")

# 6. Datentyp-Validierung
print("\n🔧 DATENTYP-VALIDIERUNG:")
type_issues = []

for i, politician in enumerate(politicians):
    if not isinstance(politician.get('neo4j_content'), list):
        type_issues.append(f"Politiker {i}: neo4j_content ist kein Array")
    
    content = politician.get('neo4j_content', [])
    for j, content_node in enumerate(content):
        if not isinstance(content_node, dict):
            type_issues.append(f"Politiker {i}, Content {j}: ist kein Objekt")

print(f"  Datentyp-Probleme: {len(type_issues)}")

# 7. Erste paar Politiker detailliert anzeigen
print("\n�� ERSTE POLITIKER IM DETAIL:")
for i in range(min(3, len(politicians))):
    p = politicians[i]
    print(f"\n  Politiker {i+1}: {p.get('full_name', 'N/A')}")
    print(f"    Element-ID: {p.get('neo4j_element_id', 'N/A')}")
    print(f"    Content-Nodes: {len(p.get('neo4j_content', []))}")
    print(f"    Verfügbare Felder: {list(p.keys())}")

# 8. Zusammenfassung
print("\n📊 ZUSAMMENFASSUNG:")
total_content_nodes = sum(len(p.get('neo4j_content', [])) for p in politicians)
print(f"  Gesamte Content-Nodes: {total_content_nodes}")
print(f"  Durchschnitt Content pro Politiker: {total_content_nodes / len(politicians):.1f}")

if content_issues:
    print(f"\n⚠️ WARNUNG: {len(content_issues)} Content-Probleme gefunden!")
    for issue in content_issues[:5]:  # Erste 5 anzeigen
        print(f"  - {issue}")

if type_issues:
    print(f"\n❌ FEHLER: {len(type_issues)} Datentyp-Probleme gefunden!")

�� Gesamtanzahl Politiker: 4455

🔍 DATENSTRUKTUR-VALIDIERUNG:
  detail_page: 4455/4455 vorhanden
  firstname: 4454/4455 vorhanden
  full_name: 4455/4455 vorhanden
  lastname: 4454/4455 vorhanden
  birth_year: 4455/4455 vorhanden
  death_year: 2012/4455 vorhanden
  neo4j_element_id: 4455/4455 vorhanden
  neo4j_content: 4455/4455 vorhanden

📊 CONTENT-VERFÜGBARKEIT:
  Politiker mit Content: 4455/4455
  Durchschnitt Content pro Politiker: 6.3
  Min/Max Content: 1 / 19

🆔 ID-INTEGRITÄT:
  Eindeutige IDs: 4455/4455
  Duplikate: 0

📋 FELD-FÜLLGRAD:
  detail_page: 4455/4455 (100.0%)
  firstname: 4454/4455 (100.0%)
  full_name: 4455/4455 (100.0%)
  lastname: 4454/4455 (100.0%)
  birth_year: 4455/4455 (100.0%)
  death_year: 2012/4455 (45.2%)
  neo4j_element_id: 4455/4455 (100.0%)
  neo4j_content: 4455/4455 (100.0%)

📝 CONTENT-QUALITÄT:
  Content-Probleme gefunden: 0

🔧 DATENTYP-VALIDIERUNG:
  Datentyp-Probleme: 0

�� ERSTE POLITIKER IM DETAIL:

  Politiker 1: Gerhart Baum
    Element-ID: 4:efc8e

In [6]:
# Politiker mit nur 1 Section filtern
politicians_with_one_section = []

for politician in politicians:
    content_count = len(politician.get('neo4j_content', []))
    if content_count == 1:
        politicians_with_one_section.append({
            'full_name': politician.get('full_name', 'N/A'),
            'element_id': politician.get('neo4j_element_id', 'N/A'),
            'content_count': content_count,
            'section_header': politician['neo4j_content'][0].get('section_header', 'N/A') if politician['neo4j_content'] else 'N/A',
            'section_content_preview': politician['neo4j_content'][0].get('section_content', '')[:100] + '...' if politician['neo4j_content'] and politician['neo4j_content'][0].get('section_content') else 'N/A'
        })

# Als DataFrame ausgeben
df_one_section = pd.DataFrame(politicians_with_one_section)

print(f"�� Politiker mit nur 1 Section gefunden: {len(df_one_section)}")
print(f"📊 Das sind {len(df_one_section) / len(politicians) * 100:.1f}% aller Politiker")

# DataFrame anzeigen
df_one_section.head(20)

# Statistiken der Section-Headers
print(f"\n📋 SECTION-HEADER VERTEILUNG:")
section_counts = df_one_section['section_header'].value_counts()
print(section_counts.head(10))

# Speichern als CSV für weitere Analyse
df_one_section.to_csv('testdata/politicians_with_one_section.csv', index=False, encoding='utf-8')
print(f"\n💾 CSV gespeichert: testdata/politicians_with_one_section.csv")

�� Politiker mit nur 1 Section gefunden: 3
📊 Das sind 0.1% aller Politiker

📋 SECTION-HEADER VERTEILUNG:
section_header
#    3
Name: count, dtype: int64

💾 CSV gespeichert: testdata/politicians_with_one_section.csv


In [ ]:
def filter_education_content(input_file='testdata/neo4j_data_politicians.json', output_file='testdata/neo4j_data_politicians_filtered.json'):
    """
    Filtert die JSON-Datei und entfernt die häufigsten Felder, die keine Bildungsinformationen enthalten
    """
    # JSON laden
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Felder die gefiltert werden sollen (nur die ursprünglichen, bewährten)
    fields_to_filter = {
        'Weblinks',           # 163x - keine Bildungsinfos
        'Einzelnachweise',    # 158x - keine Bildungsinfos
        'Literatur',          # 114x - keine Bildungsinfos
        'Abgeordneter',       # 46x - keine Bildungsinfos
        'Abgeordnete',        # 13x - keine Bildungsinfos
        'Siehe auch'          # 41x - keine Bildungsinfos
    }
    
    # Gefilterte Daten erstellen
    filtered_data = []
    total_sections_before = 0
    total_sections_after = 0
    
    for person in data:
        person_copy = person.copy()
        
        # neo4j_content filtern
        if 'neo4j_content' in person_copy:
            original_content = person_copy['neo4j_content']
            total_sections_before += len(original_content)
            
            # Nur Sections behalten, die nicht in der Filter-Liste stehen
            filtered_content = []
            for content in original_content:
                if content.get('section_header') not in fields_to_filter:
                    filtered_content.append(content)
            
            person_copy['neo4j_content'] = filtered_content
            total_sections_after += len(filtered_content)
        
        filtered_data.append(person_copy)
    
    # Gefilterte JSON speichern
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_data, f, ensure_ascii=False, indent=2)
    
    # Statistiken ausgeben
    print(f"=== FILTERUNG ERFOLGREICH ===")
    print(f"Gefilterte JSON gespeichert als: {output_file}")
    print(f"Anzahl Personen: {len(data)}")
    print(f"Gefilterte Felder: {', '.join(fields_to_filter)}")
    print(f"Section-Headers vor Filterung: {total_sections_before}")
    print(f"Section-Headers nach Filterung: {total_sections_after}")
    print(f"Entfernte Sections: {total_sections_before - total_sections_after}")
    print(f"Reduktion: {((total_sections_before - total_sections_after) / total_sections_before * 100):.1f}%")
    
    # Verbleibende Section-Headers analysieren
    remaining_headers = {}
    for person in filtered_data:
        if 'neo4j_content' in person:
            for content in person['neo4j_content']:
                header = content.get('section_header', '')
                remaining_headers[header] = remaining_headers.get(header, 0) + 1
    
    print(f"\n=== VERBLEIBENDE SECTION-HEADERS (Top 20) ===")
    sorted_headers = sorted(remaining_headers.items(), key=lambda x: x[1], reverse=True)
    for header, count in sorted_headers[:20]:
        percentage = (count / len(data)) * 100
        print(f"  {header:<40} | {count:>3}x | {percentage:>5.1f}% der Personen")

filter_education_content()

=== FILTERUNG ERFOLGREICH ===
Gefilterte JSON gespeichert als: neo4j_data_politicians_filtered.json
Anzahl Personen: 4455
Gefilterte Felder: Abgeordneter, Weblinks, Abgeordnete, Einzelnachweise, Siehe auch, Literatur
Section-Headers vor Filterung: 28173
Section-Headers nach Filterung: 17371
Entfernte Sections: 10802
Reduktion: 38.3%

=== VERBLEIBENDE SECTION-HEADERS (Top 20) ===
  #                                        | 4455x | 100.0% der Personen
  Leben                                    | 1761x |  39.5% der Personen
  Leben und Beruf                          | 1480x |  33.2% der Personen
  Partei                                   | 1214x |  27.3% der Personen
  Politik                                  | 1028x |  23.1% der Personen
  Ehrungen                                 | 742x |  16.7% der Personen
  Öffentliche Ämter                        | 528x |  11.9% der Personen
  Veröffentlichungen                       | 348x |   7.8% der Personen
  Auszeichnungen                     